In [2]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
from matplotlib import rcParams
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
# 한글 폰트 설정
rcParams['font.family'] = 'Malgun Gothic'  # 맑은 고딕으로 설정
rcParams['axes.unicode_minus'] = False     # 음수 기호 문제 해결

In [3]:
# EXCEL파일 읽기
file_path = './Text_Data/bigkinds/삼성전자_사회_20170901-20240830_전처리후.xlsx'
df_bigkinds = pd.read_excel(file_path, converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

In [4]:
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,URL,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사
0,02100101.20170901163910001,2017-09-01,매일경제,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일",사회>사건_사고,사회>교육_시험,NaN,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일 이재용,항소심,13부,...",이재용 항소심 서울고법 형사 13부에 배당 이달중 첫 기일 이재용 항소심 13부 서...,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13부 배당 이달 중 첫 기 ㄹ 이재용 항소심 13부 ...,이재 용 항소심 서울 고법 형사 13 부 배당 이달 중 첫 기일 이재 용 항소심 1...,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,이재용 항소심 서울고법 형사 부 배당 이달 중 기일 이재용 항소심 부 서울고법 형사...
1,02100101.20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...
2,02100101.20170901181209001,2017-09-01,매일경제,정주원,`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당,사회>사건_사고,사회>여성,사회>교육_시험,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당 삼성,뇌물,신설,이재용,항소심...",삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재 용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재 용 항소심...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 부 배당 삼성 뇌물 신설 이재용 항소심 형사 배...
3,02100101.20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...
4,02100311.20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,http://www.sedaily.com/NewsView/1OKUY3ELL1,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055,02100601.20240826182228001,2024-08-26,한국경제,오현우,수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다,사회>의료_건강,IT_과학>과학,NaN,NaN,NaN,...,https://www.hankyung.com/article/2024082667731,NaN,"수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다 수술,이송,재활,로봇,의료대,걱...",수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다 수술 이송 재활 로봇 의료대 걱...,수술 이송 재활 다 하는 로봇 의료 대란 걱정 덜었다 수술 이송 재활 로봇 의료 대...,수술 이송 재활 다 는 로봇 의료대란 걱정 덜 었다 수술 이송 재활 로봇 의료대 걱...,수술 이송 재활 다 는 로봇 의료 대란 걱정 덜 었 다 수술 이송 재활 로봇 의료 ...,수술 이송 재활 다 는 로봇 의료 대란 걱정 덜 었 다 수술 이송 재활 로봇 의료 ...,수술 이송 재활 로봇 의료 대란 걱정 수술 이송 재활 로봇 의료 걱정 고령 필연 의...,수술 이송 재활 로봇 의료 대란 걱정 수술 이송 재활 로봇 의료 대 걱정 고령 필연...
6056,02100601.20240826161718002,2024-08-26,한국경제,이해성,"삼성전자 기흥 피폭 작업자, 손에 기준치의 188배 피폭",사회>사회일반,IT_과학>과학,NaN,NaN,NaN,...,https://www.hankyung.com/article/202408266335i,NaN,"삼성전자 기흥 피폭 작업자, 손에 기준치의 188배 피폭 삼성전자,기흥,피폭,작업자...",삼성전자 기흥 피폭 작업자

In [5]:
# KoBERT 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("kwoncho/KoFinBERT")
model = AutoModelForSequenceClassification.from_pretrained("kwoncho/KoFinBERT")

In [6]:
text_classifier = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    truncation=True,  # 긴 텍스트를 자동으로 자름
    max_length=512,   # 최대 길이를 512로 제한
    top_k=None,       # 모든 레이블의 확률 반환
    device=0          # GPU 사용 가능 시 (0), 그렇지 않으면 CPU (-1)
)

In [7]:
max_length = 512

s = time.time()
predicted_scores = []
# tqdm을 사용하여 처리 상태 표시
for text in tqdm(df_bigkinds['제목_키워드_전처리후_Okt'], desc="Predicting"):
    # 감정 분석 수행
    prediction = text_classifier(text)
    
    # 레이블을 숫자 값으로 매핑
    label_mapping = {
        'positive': 1,
        'neutral': 0,
        'negative': -1
    }
    
    # 연속적인 점수 직접 계산
    score = 0
    for item in prediction[0]:
        label = item['label']
        prob = item['score']
        score += label_mapping[label] * prob
    predicted_scores.append(score)

print("Prediction Time: ", time.time() - s)

Predicting: 100%|██████████████████████████████████████████████████████████████████| 6060/6060 [03:13<00:00, 31.26it/s]

Prediction Time:  193.85333347320557


In [8]:
# 뉴스 식별자 컬럼에서 '.' 이후의 값만 조회
df_bigkinds['뉴스 식별자'] = df_bigkinds['뉴스 식별자'].str.split('.').str[1]

In [9]:
# 데이터프레임에 결과 추가
df_bigkinds['KoFinBERT_Sentiment'] = predicted_scores
# 소수점 1자리로 반올림
df_bigkinds['KoFinBERT_Sentiment'] = df_bigkinds['KoFinBERT_Sentiment'].round(1)
df_bigkinds['KoFinBERT_Sentiment'] = df_bigkinds['KoFinBERT_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사,KoFinBERT_Sentiment
0,20170901163910001,2017-09-01,매일경제,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일",사회>사건_사고,사회>교육_시험,NaN,범죄>정치>뇌물수수,NaN,...,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일 이재용,항소심,13부,...",이재용 항소심 서울고법 형사 13부에 배당 이달중 첫 기일 이재용 항소심 13부 서...,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13부 배당 이달 중 첫 기 ㄹ 이재용 항소심 13부 ...,이재 용 항소심 서울 고법 형사 13 부 배당 이달 중 첫 기일 이재 용 항소심 1...,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,이재용 항소심 서울고법 형사 부 배당 이달 중 기일 이재용 항소심 부 서울고법 형사...,0.0
1,20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...,0.0
2,20170901181209001,2017-09-01,매일경제,정주원,`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당,사회>사건_사고,사회>여성,사회>교육_시험,범죄>정치>뇌물수수,NaN,...,NaN,"`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당 삼성,뇌물,신설,이재용,항소심...",삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재 용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재 용 항소심...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 부 배당 삼성 뇌물 신설 이재용 항소심 형사 배...,0.0
3,20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...,0.0
4,20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055,20240826182228001,2024-08-26,한국경제,오현우,수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다,사회>의료_건강,IT_과학>과학,NaN,NaN,NaN,...,NaN,"수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다 수술,이송,재활,로봇,의료대,걱...",수술 이송 재활 다 하는 로봇 의료대란 걱정 덜었다 수술 이송 재활 로봇 의료대 걱...,수술 이송 재활 다 하는 로봇 의료 대란 걱정 덜었다 수술 이송 재활 로봇 의료 대...,수술 이송 재활 다 는 로봇 의료대란 걱정 덜 었다 수술 이송 재활 로봇 의료대 걱...,수술 이송 재활 다 는 로봇 의료 대란 걱정 덜 었 다 수술 이송 재활 로봇 의료 ...,수술 이송 재활 다 는 로봇 의료 대란 걱정 덜 었 다 수술 이송 재활 로봇 의료 ...,수술 이송 재활 로봇 의료 대란 걱정 수술 이송 재활 로봇 의료 걱정 고령 필연 의...,수술 이송 재활 로봇 의료 대란 걱정 수술 이송 재활 로봇 의료 대 걱정 고령 필연...,0.0
6056,20240826161718002,2024-08-26,한국경제,이해성,"삼성전자 기흥 피폭 작업자, 손에 기준치의 188배 피폭",사회>사회일반,IT_과학>과학,NaN,NaN,NaN,...,NaN,"삼성전자 기흥 피폭 작업자, 손에 기준치의 188배 피폭 삼성전자,기흥,피폭,작업자...",삼성전자 기흥 피폭 작업자 손에 기준치의 188배 피폭 삼성전자 기흥 피폭 작업자 ...,삼성 기흥 피폭 작업 손 기준 치의 188 배 피폭 삼성 기흥 피폭 작업 기 준치 ...,삼성전자 기흥 피폭 작업자 손 기준치 188배 피폭 삼성전자 기흥 피폭 작업자 기준...,삼성전자 기흥 피폭 작업자 손 기준치 188 배 피폭 삼성전자 기흥 피폭 작업자 기...,삼성전자 기흥 피폭 작업자 손 기준 치 188 배 피폭 삼성전자 기흥 피폭 작업자 ...,삼성전자 기흥 피폭 작업자 기준 188 피폭 삼성전자 기흥 피폭 작업자 기준 피폭 ...,삼성전자 기흥 피폭 작업자 손 기준 배 피폭 삼성전자 기흥 피폭 작업자 기준 피폭 ...,0.0
6057,20240827181505

전체 분석 후 분석제외 대상과 중복건 제거

In [11]:
df_bigkinds = df_bigkinds[df_bigkinds['분석제외 여부'].isnull()]

In [12]:
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사,KoFinBERT_Sentiment
0,20170901163910001,2017-09-01,매일경제,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일",사회>사건_사고,사회>교육_시험,NaN,범죄>정치>뇌물수수,NaN,...,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일 이재용,항소심,13부,...",이재용 항소심 서울고법 형사 13부에 배당 이달중 첫 기일 이재용 항소심 13부 서...,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13부 배당 이달 중 첫 기 ㄹ 이재용 항소심 13부 ...,이재 용 항소심 서울 고법 형사 13 부 배당 이달 중 첫 기일 이재 용 항소심 1...,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,이재용 항소심 서울고법 형사 부 배당 이달 중 기일 이재용 항소심 부 서울고법 형사...,0.0
1,20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...,0.0
2,20170901181209001,2017-09-01,매일경제,정주원,`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당,사회>사건_사고,사회>여성,사회>교육_시험,범죄>정치>뇌물수수,NaN,...,NaN,"`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당 삼성,뇌물,신설,이재용,항소심...",삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재 용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재 용 항소심...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 부 배당 삼성 뇌물 신설 이재용 항소심 형사 배...,0.0
3,20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...,0.0
4,20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6053,20240823175054001,2024-08-23,한국경제,신정은/김형규,현대차가 쏘아올린 산업계 '재고용 바람',경제>자동차,경제>산업_기업,사회>노동_복지,NaN,NaN,...,NaN,"현대차가 쏘아올린 산업계 '재고용 바람' 현대차,바람,산업,정년,생산,근로자,재고용...",현대차가 쏘아올린 산업계 재고용 바람 현대차 바람 산업 정년 생산 근로자 재고용 움...,현대차 쏘아 올린 산업 계 재고용 바람 현대차 바람 산업 정년 생산 근로자 재고용 ...,현대차 쏘아올린 산업계 재고용 바라 ㅁ 현대차 바라 ㅁ 산업 정년 생산 근로자 재고...,현대 차 쏘 올리 ㄴ 산 업계 재고용 바람 현대 차 바람 산업 정년 생산 근로자 재...,현대차 쏘 올리 ㄴ 산업 계 재 고용 바람 현대차 바람 산업 정년 생산 근로자 재 ...,현대차 산업 고용 바람 현대차 바람 산업 정년 생산 근로자 재고 움직임 자동차업 전...,현대차 산업 고용 바람 현대차 바람 산업 정년 생산 근로자 고용 움직임 자동차업 전...,0.0
6054,20240825142511001,2024-08-25,매일경제,박재영 기자(jyp8909@mk.co.kr),"부산국제마케팅광고제, 올해 그랑프리 수상작 발표",사회>사회일반,NaN,NaN,NaN,NaN,...,NaN,"부산국제마케팅광고제, 올해 그랑프리 수상작 발표 부산국제마케팅광고제,그랑프리,수상작...",부산국제마케팅광고제 올해 그랑프리 수상작 발표 부산국제마케팅광고제 그랑프리 수상작 ...,부산 국제 마케팅 광고 올해 그랑프리 수상작 발표 부산 국제 마케팅 광고 그랑프리 ...,부산국제마케팅광고제 올해 그랑프리 수상작 발표 부산국제마케팅광고제 그랑프리 수상작 ...,부산 국제 마케팅 광고 올해 그랑프리 수상작 발표 부산 국제 마케팅 광고 그랑프리 ...,부산 국제 마케팅 광고 올해 그랑프리 수상작 발표 부산 국제 마케팅 광고 그랑프리 ...,부산 국제 마케팅 광고제 올해 그랑프리 수상작 발표 부산 국제 마케팅 광고제 그랑프...,부산 국제 마케팅 광고 올해 그랑프리 수상작 발표 부산 국제 마케팅 광고 그랑프리 ...,0.0
6055,20

뉴스 식별자 최소갑 기준으로 제목_키워드_전처리후 그룹핑

In [14]:
# 그룹핑할 컬럼 리스트
group_columns = ['제목_키워드_전처리후_Okt']

# 각 그룹에서 뉴스 식별자 컬럼의 최소값을 유지
df_bigkinds2 = df_bigkinds.groupby(group_columns, as_index=False).agg({'뉴스 식별자': 'min'})

In [15]:
df_bigkinds2

,제목_키워드_전처리후_Okt,뉴스 식별자
0,- 3 주 째 대면 불발 에도 이재용 기소 조만간 결정 날 듯 3 주 대면 불발 이...,20200715150951001
1,- TV PD 수첩 암페타민 밀 반입 박봄 은 입건 유예 됐나 PD 수첩 박봄 암페...,20180425075416001
2,- 삼성 26일 수사 심의 위 화 력 집중 삼성 수사 심의 위 화 력 집중 부회장 ...,20200625175758001
3,1 078일 만 실형 선고 이재용 정면 응시 한 채 한동안 깊은 침묵 1 078일 ...,20210118151417001
4,1 노총 잃은 한국 노총 정부 정책 협약 재 검토 1 노총 한국 노총 정부 재 검토...,20200107211129001
...,...,...
5475,흔들리는 직장인 10 명 중 7 명 이직 퇴사 고민 직장인 10 명 이직 퇴사 고민...,20210829174514001
5476,희 대의 사기 큰손 장영자 아시나요 번째 사기 징역 4년 확정 희대 사기 큰손 장영...,20200409114505001
5477,희귀 병 딸 도와준 이재용 사칭 계정 사건 전말 알 고 보니 계정 이재용 사칭 사건...,20230612114647001
5478,희망 성화 봉송 로봇다리 수영 선수 김세진 참여 희망 성화 봉송 로봇다리 수영 선수...,20171112074911001


In [16]:
# 필요한 컬럼만 선택하여 두 데이터프레임 병합
big_kinds_sentiment = pd.merge(
    df_bigkinds[['일자','제목_키워드_전처리후_Okt', '뉴스 식별자','KoFinBERT_Sentiment']],
    df_bigkinds2[['제목_키워드_전처리후_Okt', '뉴스 식별자']],
    on=['제목_키워드_전처리후_Okt', '뉴스 식별자'],
    how='inner'
)
big_kinds_sentiment = big_kinds_sentiment[['뉴스 식별자','일자','제목_키워드_전처리후_Okt','KoFinBERT_Sentiment']]
big_kinds_sentiment

,뉴스 식별자,일자,제목_키워드_전처리후_Okt,KoFinBERT_Sentiment
0,20170901163910001,2017-09-01,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,0.0
1,20170901174354001,2017-09-01,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,0.0
2,20170901181209001,2017-09-01,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,0.0
3,20170901141935001,2017-09-01,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,0.0
4,20170901153936001,2017-09-01,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,-0.1
...,...,...,...,...
5475,20240823175054001,2024-08-23,현대차 쏘아 올린 산업 계 재고용 바람 현대차 바람 산업 정년 생산 근로자 재고용 ...,0.0
5476,20240825142511001,2024-08-25,부산 국제 마케팅 광고 올해 그랑프리 수상작 발표 부산 국제 마케팅 광고 그랑프리 ...,0.0
5477,20240826182228001,2024-08-26,수술 이송 재활 다 하는 로봇 의료 대란 걱정 덜었다 수술 이송 재활 로봇 의료 대...,0.0
5478,20240826161718002,2024-08-26,삼성 기흥 피폭 작업 손 기준 치의 188 배 피폭 삼성 기흥 피폭 작업 기 준치 ...,0.0


In [17]:
big_kinds_sentiment.to_excel('./Text_Data/bigkinds/삼성전자_사회_20170901-20240830_전처리후_Okt_KoFinBERT_id.xlsx', index=False)

In [40]:
big_kinds_sentiment_daily = big_kinds_sentiment.groupby('일자', as_index=False)['KoFinBERT_Sentiment'].mean()
# 소수점 1자리로 반올림
big_kinds_sentiment_daily['KoFinBERT_Sentiment'] = big_kinds_sentiment_daily['KoFinBERT_Sentiment'].round(1)
big_kinds_sentiment_daily['KoFinBERT_Sentiment'] = big_kinds_sentiment_daily['KoFinBERT_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
big_kinds_sentiment_daily

,일자,KoFinBERT_Sentiment
0,2017-09-01,0.0
1,2017-09-04,-0.7
2,2017-09-05,-0.2
3,2017-09-06,0.0
4,2017-09-07,0.0
...,...,...
1646,2024-08-22,0.0
1647,2024-08-23,0.0
1648,2024-08-25,0.0
1649,2024-08-26,0.0


In [42]:
big_kinds_sentiment_daily = big_kinds_sentiment_daily.rename(columns={'일자': 'date', 'KoFinBERT_Sentiment': 'sentiment'})

In [44]:
big_kinds_sentiment_daily

,date,sentiment
0,2017-09-01,0.0
1,2017-09-04,-0.7
2,2017-09-05,-0.2
3,2017-09-06,0.0
4,2017-09-07,0.0
...,...,...
1646,2024-08-22,0.0
1647,2024-08-23,0.0
1648,2024-08-25,0.0
1649,2024-08-26,0.0


In [46]:
big_kinds_sentiment_daily.to_excel('./Text_Data/bigkinds/삼성전자_사회_20170901-20240830_전처리후_Okt_KoFinBERT_id_daily.xlsx', index=False)